In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

import datetime

In [2]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [3]:
import argparse

parser = argparse.ArgumentParser(description = "Please insert the train flag")
parser.add_argument('-t', '--train', action = "store",
                    help='If true, we train and save. Else, otherwise.', required = True)

my_args = vars(parser.parse_args())
trainFlag = my_args['train']
trainFlag = trainFlag.lower() in ("True", "t", "true", "1", 1)

usage: ipykernel_launcher.py [-h] -t TRAIN
ipykernel_launcher.py: error: the following arguments are required: -t/--train


SystemExit: 2

/home/nabeel/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
logging = tf.logging

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
print(datetime.datetime.now())

2017-08-27 21:18:47.906131


In [7]:
df = np.load('invokerItems.npy')
match_ids = pd.read_pickle('final_processed_invoker_data')['matchId']

x = np.random.rand(df.shape[0])
mask = np.where(x < 0.75)[0]
mask2 = np.where(x >= 0.75)[0]
df_train = df[mask, :]
df_test = df[mask2, :]
match_ids_train = match_ids.iloc[mask]
match_ids_test = match_ids.iloc[mask2]

In [8]:
# 265 is highest ID, and we need a zero for empty
NUM_ITEMS = 266
DIM = NUM_ITEMS

batchSize = int(len(df) * 1.0 /20)
batched_ItemData = tf.placeholder(tf.float32, shape=[None, None, DIM])

batched_ItemData = tf.pad(batched_ItemData, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_ItemData_strided = tf.strided_slice(batched_ItemData, [0, 1, 0], tf.shape(batched_ItemData))
targets = tf.pad(batched_ItemData_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
MAX_SequenceLength = df.shape[1]
batched_ItemDataSequenceLengths = getSequenceLengths(batched_ItemData)
lstmHiddenSize = 50
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmHiddenSize)

In [9]:
state = tf.Variable(lstmCell.zero_state(batchSize, tf.float32), trainable=False, name='lstm_state')

lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=lstmCell,
    dtype=tf.float32,
    sequence_length=batched_ItemDataSequenceLengths,
    inputs=batched_ItemData,
    initial_state = tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]))

# the assign is run every time we run on lstmOutput because of the with
with tf.control_dependencies([state.assign(last_states)]):
    lstmOutput = tf.identity(lstmOutput)

In [10]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [11]:
with tf.variable_scope('LayerToLogits'):
    weights = tf.Variable(tf.random_normal([lstmHiddenSize, NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='weightsLayer')
    bias = tf.Variable(tf.random_normal([NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='biasLayer')

In [24]:
logits = tf.matmul(lstmOutput, weights) + bias
targets = tf.reshape(targets, [-1, DIM])

with tf.variable_scope('accuracy'):
    with tf.variable_scope('correct_prediction'):
        correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
        correct_predictions = tf.reshape(correct_predictions, shape=[batchSize, MAX_SequenceLength])
        correct_predictions = tf.cast(correct_predictions, tf.float32)
    with tf.variable_scope('accuracy'):
        accuracyPerSequence = tf.reduce_sum(correct_predictions, reduction_indices=1) / \
            tf.cast(batched_ItemDataSequenceLengths, tf.float32)
        averageAccuracy =  tf.reduce_mean(accuracyPerSequence)

loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = logits)
mask = tf.sign(tf.reduce_max(tf.abs(targets), 1))
loss = mask * loss
loss = tf.reshape(loss, shape=[batchSize, MAX_SequenceLength])
# loss is 2d in a dynamic length way and then make it 1d to then reduce on
loss_per_sequence = tf.reduce_sum(loss, reduction_indices=1) / tf.cast(batched_ItemDataSequenceLengths, tf.float32)
cost = tf.reduce_mean(loss_per_sequence)

In [25]:
# default of 0.001
learning_rate = 0.001
# default of 0.99
beta1 = 0.99
# default of 0.999
beta2 = 0.999
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1, beta2 = beta2)

global_step = tf.Variable(0, name='global_step', trainable=False)

gradients, variables = zip(*optimizer.compute_gradients(cost))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
train_op = optimizer.apply_gradients(zip(gradients, variables))

** Generate summary information **

In [26]:
# code from tensorflow documentation
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
with tf.name_scope('weights'):
    variable_summaries(weights)
    
with tf.name_scope('bias'):
    variable_summaries(bias)

with tf.name_scope("loss"):
    variable_summaries(loss_per_sequence)
    # this number is already calculated in variable_summaries of loss_per_sequence but be explicit
    tf.summary.scalar('cost', cost)
    
with tf.name_scope("accuracy"):
    variable_summaries(accuracyPerSequence)
    # this number is already calculated in variable_summaries of accuracyPerSequence but be explicit
    tf.summary.scalar('accuracy', averageAccuracy)

summary_op = tf.summary.merge_all()

In [32]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
ckpoint_dir = os.path.join(os.getcwd(), 'model-backups/model.ckpt')
print(ckpoint_dir)

/home/nabeel/Documents/Projects/Dota-ItemSequence/model-backups/model.ckpt


In [16]:
def train(sess):
    numEpochs = 1000
    numBatches = 20
    for epochIter in range(numEpochs):
        print('Epoch: {0}'.format(epochIter))
        for batchItr in range(numBatches):
            samp = np.random.choice(np.arange(df_train.shape[0]), size=batchSize, replace=True, p=None)
            if (epochIter+1) % 50 == 0:
                summaries, _ = sess.run([summary_op, train_op], 
                                         feed_dict = {batched_ItemData : df_train[samp, :, :]})
                train_writer.add_summary(summaries, epochIter)
                # plus 1 to get a more readable number is tensorboard
                saver.save(sess, ckpoint_dir, global_step = global_step)
            else:
                sess.run(train_op, feed_dict = {batched_ItemData : df_train[samp, :, :]})

In [27]:
def test(sess):
    summaries, averageAccuracy_val, cost_val = sess.run([summary_op, averageAccuracy, cost], 
                                         feed_dict = {batched_ItemData : df_test})
    print('Accuracy: {0}'.format(averageAccuracy_val))
    print('Loss: {0}'.format(cost_val))
    test_writer.add_summary(summaries)

In [28]:
def encodeItem(itemId):
    return [int(i==int(itemId)) for i in range(NUM_ITEMS)]

def generate(sess, initialItems, initialState = state):
    pass

In [35]:
trainFlag = True
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('logs/train',
                                      sess.graph)
    test_writer = tf.summary.FileWriter('logs/test')
    if (trainFlag):
        sess.run(init)
        train(sess)
    else:
        print('Doing test')
        saver.restore(sess, ckpoint_dir)
        test(sess)

Doing test
INFO:tensorflow:Restoring parameters from /home/nabeel/Documents/Projects/Dota-ItemSequence/model-backups/model.ckpt


NotFoundError: Key rnn/basic_lstm_cell/kernel/Adam_3 not found in checkpoint
	 [[Node: save_2/RestoreV2_26 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_2/Const_0_0, save_2/RestoreV2_26/tensor_names, save_2/RestoreV2_26/shape_and_slices)]]

Caused by op 'save_2/RestoreV2_26', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nabeel/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nabeel/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/nabeel/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/nabeel/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py